In [3]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import openai
from string import Template
import json
from neo4j import GraphDatabase
import glob
from timeit import default_timer as timer
from dotenv import load_dotenv
from time import sleep
import re
from pathlib import Path
import csv
import pandas as pd

KeyboardInterrupt: 

: 

In [72]:
# Load environment variables
load_dotenv('.env')

True

In [73]:
# Neo4j configuration & constraints
neo4j_url = os.getenv("NEO4J_CONNECTION_URL")
neo4j_user = os.getenv("NEO4J_USER")
neo4j_password = os.getenv("NEO4J_PASSWORD")
gds = GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password))

In [74]:
raw_csv_path = "amazon_products/cosmetics_25_products.csv"

In [75]:


openai_deployment = "gpt-4"
client = openai.OpenAI()

def extract_json_block(text):
    match = re.search(r"\{[\s\S]*\}", text)
    if match:
        return match.group(0)
    return None

def process_gpt(prompt, system_msg):
    completion = client.chat.completions.create(
        model=openai_deployment,
        temperature=0,
        max_tokens=2000,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": prompt}
        ],
    )
    
    sleep(0.5)
    
    return completion.choices[0].message.content

def extract_entities_relationships_from_csv(csv_file, prompt_template):
    system_msg = "You are a beauty and personal care expert who understands structured product relationships."
    results = []
    MAX_INGREDIENTS = 10
    MAX_DESCRIPTION_LENGTH = 500

    # Use pandas to safely load the CSV
    df = pd.read_csv(csv_file, quotechar='"', escapechar='\\', engine='python', encoding='utf-8-sig')

    for _, row in df.iterrows():
        try:
            ingredients = row.get("Ingredients", "")
            if ingredients.count(",") > MAX_INGREDIENTS:
                ingredients = ",".join(ingredients.split(",")[:MAX_INGREDIENTS]) + "..."
            prompt = Template(prompt_template).substitute(
                label=row.get("Label", ""),
                brand=row.get("Brand", ""),
                name=row.get("Name", ""),
                price=row.get("Price", ""),
                rank=row.get("Rank", ""),
                ingredients=ingredients,
                for_skin_type=row.get("For_Skin_Type", ""),
                size=row.get("Size", ""),
                description=row.get("Description", ""),
            )

            result = process_gpt(prompt, system_msg)
            json_str = extract_json_block(result)
            print("--- Extracted JSON Block ---")
            print(json_str)
            if json_str:
                results.append(json.loads(json_str))

            print(f"Successfully executed row {row.get('Name', '')}")

        except Exception as e:
            print(f"Failed on row {row.get('Name', '')}: {e}")
            continue

    return results

In [ ]:

def normalize_id(name):
    """Create a consistent alphanumeric ID for any entity."""
    return re.sub(r'[^a-z0-9]', '_', name.lower()).strip('_')

def generate_cypher(json_obj):
    e_statements = []
    r_statements = []
    e_label_map = {}
    skipped_relationships = []

    for i, obj in enumerate(json_obj):
        print(f"Generating cypher for file {i + 1} of {len(json_obj)}")

        for entity in obj.get("entities", []):
            label = entity.get("label")
            raw_id = entity.get("id")
            entity_id = normalize_id(raw_id)

            # Normalize and set properties (excluding label/id)
            properties = {k: v for k, v in entity.items() if k not in ["label", "id"]}
            prop_str = ", ".join([f'n.{k} = {json.dumps(v)}' for k, v in properties.items()])

            cypher = f'MERGE (n:{label} {{id: "{entity_id}"}})'
            if properties:
                cypher += f' ON CREATE SET {prop_str}'

            e_statements.append(cypher)
            e_label_map[entity_id] = label  # Map normalized ID to label

        for rs in obj.get("relationships", []):
            try:
                parts = rs.split("|")
                if len(parts) != 3:
                    raise ValueError(f"Invalid relationship format: {rs}")
                src_raw, rel_type, tgt_raw = parts
                src_id = normalize_id(src_raw)
                tgt_id = normalize_id(tgt_raw)

                if src_id in e_label_map and tgt_id in e_label_map:
                    src_label = e_label_map[src_id]
                    tgt_label = e_label_map[tgt_id]
                    cypher = (
                        f'MATCH (a:{src_label} {{id: "{src_id}"}}) '
                        f'MATCH (b:{tgt_label} {{id: "{tgt_id}"}}) '
                        f'MERGE (a)-[:{rel_type}]->(b)'
                    )
                    r_statements.append(cypher)
                else:
                    skipped_relationships.append(rs)
            except Exception as e:
                print(f"Error processing relationship {rs}: {e}")
                skipped_relationships.append(rs)

    if skipped_relationships:
        print("\n Warning: Skipped relationships due to missing entities:")
        for rs in skipped_relationships:
            print(f"- {rs}")
        with open("skipped_relationships.txt", "w") as f:
            f.write("\n".join(skipped_relationships))

    with open("cyphers.txt", "w") as outfile:
        outfile.write("\n".join(e_statements + r_statements))

    return e_statements + r_statements


In [77]:
relationship_prompt = """
Given the following structured data for a beauty product, extract all relevant entities and relationships in JSON format with keys "entities" and "relationships".

Ensure:
- All entity "label" fields use valid Cypher label names: no spaces, use PascalCase or snake_case (e.g., SkinType instead of "Skin Type").
- All "id" fields are unique and consistent across entities and relationships.
- Use relationship types in ALL_CAPS_WITH_UNDERSCORES, and entity labels in PascalCase or snake_case.

Data:
{
  "label": "$label",
  "brand": "$brand",
  "name": "$name",
  "price": "$price",
  "rank": "$rank",
  "ingredients": "$ingredients",
  "for_skin_type": "$for_skin_type",
  "size": "$size",
  "description": "$description"
}

Format:
{
  "entities": [
    {"id": "unique_product_id", "label": "Product", "name": "$name", "brand": "$brand", "price": "$price", "rank": "$rank", "size": "$size"},
    {"id": "ingredient_id", "label": "Ingredient", "name": "Ingredient Name"},
    {"id": "skin_type_id", "label": "SkinType", "name": "Dry"},
    ...
  ],
  "relationships": [
    {"source": "unique_product_id", "relationship": "CONTAINS", "target": "ingredient_id"},
    {"source": "unique_product_id", "relationship": "SUITABLE_FOR", "target": "skin_type_id"},
    ...
  ]
}
"""


In [78]:
relationship_prompt = """
From the product description below, extract Entities & Relationships in an open format. You are free to:

1. Create any entity types you identify, with relevant attributes. Each entity must include:
    - label: The type of entity (e.g., 'Product', 'Ingredient', 'Benefit', etc.)
    - id: A globally unique, clean identifier derived from the entity's main name or value.
    - Any useful attributes (e.g., name, brand, function, effect, category, texture, step, price, rating, size, etc.)

2. Create relationships between:
    - Entities in this description
    - Referenced entities (other products, brands, steps, concerns, etc.)
    - For relationships between product entities, use relationship types like:
        - SAME_BRAND_AS, SIMILAR_PRICE_TIER_AS, SHARES_INGREDIENT_WITH, ADDRESSES_SAME_CONCERN, USED_IN_SAME_STEP, COMPLEMENTS, ALTERNATIVE_TO, PART_OF_COLLECTION, etc.

3. Output format MUST be valid JSON:
{
  "entities": [
    {"label": "EntityType", "id": "unique_id", ...attributes...},
    ...
  ],
  "relationships": [
    "source_id|RELATIONSHIP_TYPE|target_id",
    ...
  ]
}

Guidelines:
- All entity `id` values must be lowercase, alphanumeric strings derived from the entity's name/value (spaces and punctuation replaced with underscores).
- All entity "label" fields use valid Cypher label names: no spaces, use PascalCase or snake_case (e.g., SkinType instead of "Skin Type").
- Use relationship types in ALL_CAPS_WITH_UNDERSCORES.
- Do NOT include hardcoded examples or unrelated product names.
- Avoid duplication: if multiple entities refer to the same concept (e.g. “Water”), use the same `id`.
- Each relationship must reference entities defined in the `entities` list.
- Merge entity data by `id` to avoid partial duplicates.

Now process this structured input:

{
  "label": "$label",
  "brand": "$brand",
  "name": "$name",
  "price": "$price",
  "rank": "$rank",
  "ingredients": "$ingredients",
  "for_skin_type": "$for_skin_type",
  "size": "$size",
  "description": "$description"
}
"""


In [79]:
def run_csv_ingestion(csv_path):
    json_obj = extract_entities_relationships_from_csv(csv_path, relationship_prompt)
    cypher_statements = generate_cypher(json_obj)

    for stmt in cypher_statements:
        print(f"Running: {stmt}")
        # Execute the Cypher statement in your Neo4j database
        # Example: gds.execute_query(stmt)

run_csv_ingestion("amazon_products/cosmetics_25_products.csv")

--- Extracted JSON Block ---
{
  "entities": [
    {
      "label": "Product",
      "id": "facial_treatment_essence",
      "brand": "SK-II",
      "name": "Facial Treatment Essence",
      "price": "179",
      "rank": "4.1",
      "size": "5.0 oz (150ml)",
      "category": "Moisturizer"
    },
    {
      "label": "Ingredient",
      "id": "galactomyces_ferment_filtrate",
      "name": "Galactomyces Ferment Filtrate (Pitera)"
    },
    {
      "label": "Ingredient",
      "id": "butylene_glycol",
      "name": "Butylene Glycol"
    },
    {
      "label": "Ingredient",
      "id": "pentylene_glycol",
      "name": "Pentylene Glycol"
    },
    {
      "label": "Ingredient",
      "id": "water",
      "name": "Water"
    },
    {
      "label": "Ingredient",
      "id": "sodium_benzoate",
      "name": "Sodium Benzoate"
    },
    {
      "label": "Ingredient",
      "id": "methylparaben",
      "name": "Methylparaben"
    },
    {
      "label": "Ingredient",
      "id": "sorbic_a

In [80]:
def execute_cypher_from_file(filepath):
    try:
        with open(filepath, 'r') as file:
            cypher_statements = [line.strip() for line in file if line.strip()]

        print(f"Found {len(cypher_statements)} Cypher statements to execute")

        failed_statements = []
        with gds.session() as session:
            for i, stmt in enumerate(cypher_statements, 1):
                try:
                    print(f"Executing statement {i} of {len(cypher_statements)}")
                    session.run(stmt)
                except Exception as e:
                    print(f"Error executing statement {i}: {e}")
                    failed_statements.append((stmt, str(e)))

        success_count = len(cypher_statements) - len(failed_statements)
        print(f"\nExecution complete: {success_count}/{len(cypher_statements)} successful")

        if failed_statements:
            with open("failed_statements.txt", "w") as f:
                for stmt, error in failed_statements:
                    f.write(f"Statement: {stmt}\nError: {error}\n\n")
            print("Failed statements written to 'failed_statements.txt'")

    except FileNotFoundError:
        print(f"Error: Could not find file '{filepath}'")
    except Exception as e:
        print(f"Unexpected error: {e}")
    finally:
        gds.close()

In [81]:
execute_cypher_from_file("cyphers.txt")

Found 657 Cypher statements to execute
Executing statement 1 of 657
Executing statement 2 of 657
Executing statement 3 of 657
Executing statement 4 of 657
Executing statement 5 of 657
Executing statement 6 of 657
Executing statement 7 of 657
Executing statement 8 of 657
Executing statement 9 of 657
Executing statement 10 of 657
Executing statement 11 of 657
Executing statement 12 of 657
Executing statement 13 of 657
Executing statement 14 of 657
Executing statement 15 of 657
Executing statement 16 of 657
Executing statement 17 of 657
Executing statement 18 of 657
Executing statement 19 of 657
Executing statement 20 of 657
Executing statement 21 of 657
Executing statement 22 of 657
Executing statement 23 of 657
Executing statement 24 of 657
Executing statement 25 of 657
Executing statement 26 of 657
Executing statement 27 of 657
Executing statement 28 of 657
Executing statement 29 of 657
Executing statement 30 of 657
Executing statement 31 of 657
Executing statement 32 of 657
Executing 

In [45]:
df = pd.read_csv(raw_csv_path, quotechar='"', escapechar='\\', engine='python', encoding='utf-8-sig')
df = df.fillna("").head(2)
df

,Unnamed: 0,Label,Brand,Name,Price,Rank,Ingredients,For Skin Type,Size,Description
0,1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...","['Dry', 'Normal', 'Oily', 'Sensitive']",5.0 oz (150ml),"A signature essence with 90% Pitera, a bio-ing..."
1,3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...","['Dry', 'Normal', 'Oily', 'Sensitive']",1.0 oz (30ml),A silky-soft cream powered by miracle broth th...
